In [5]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models
from torchvision import transforms

from pathlib import Path
import numpy as np

import models.models
import models.mobilenet
from dataloaders.nyu import NYUDataset
from utils.train import train

# CIFAR pretrain

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(228)])

imagenet_data = torchvision.datasets.CIFAR100('/DATA/vashchilko/', train=True, transform=transform)

In [7]:
net = models.mobilenet.MobileNet()
lr = 0.001
criterion = nn.CrossEntropyLoss()
batch_size = 64
epochs = 1
device = torch.device('cuda:1')

In [8]:
trained_net, log = train(imagenet_data, net=net, criterion=criterion,
                    batch_size=batch_size, lr=lr,
                    epochs=epochs, device=device)

epoch 0|0; total loss:0.058855986293358135
last losses: 4.649622917175293
epoch 0|79; total loss:4.3774337466759015
last losses: 4.2719550132751465
epoch 0|158; total loss:3.988340957255303
last losses: 3.9062016010284424
epoch 0|237; total loss:3.756196335901188
last losses: 3.9489104747772217
epoch 0|316; total loss:3.6155811382245413
last losses: 3.572338819503784
epoch 0|395; total loss:3.425937679749501
last losses: 3.6279380321502686
epoch 0|474; total loss:3.325775475441655
last losses: 3.4114978313446045
epoch 0|553; total loss:3.2321295949477182
last losses: 3.1573095321655273
epoch 0|632; total loss:3.1067257350004174
last losses: 3.09793758392334
epoch 0|711; total loss:2.996826959561698
last losses: 2.817779302597046
Finished Training


In [9]:
torch.save(trained_net, './outputs/cifar100.pth')

# Training on NYUv2

In [10]:
base_nyu = Path("/DATA/vashchilko/nyudepthv2")
traindir_nyu = base_nyu / 'train'
valdir_nyu = base_nyu / 'val'

assert traindir_nyu.exists(), "Set your own path to train"
assert valdir_nyu.exists(), "Set your own path to val"

In [11]:
train_dataset = NYUDataset(traindir_nyu, split='train')

In [13]:
net = models.models.Model(pretrained=True)
lr = 0.001
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0001)
criterion = nn.MSELoss()
batch_size = 8
epochs = 2
device = torch.device('cuda:1')

In [14]:
%%time
trained_net, log = train(train_dataset, net=net, criterion=criterion,
                    batch_size=batch_size, lr=lr,
                    epochs=epochs, device=device)

epoch 0|0; total loss:0.004015871862164971
last losses: 2.361332654953003
epoch 0|588; total loss:3.0731045265181534
last losses: 1.5620306730270386


KeyboardInterrupt: 

In [15]:
torch.save(trained_net.state_dict(), './outputs/trained3_state.pth')
np.save('./ouputs/log1.npy', np.array(log))

NameError: name 'np' is not defined

In [ ]:
# add graphics for training: loss, metrics, add func for this into utils.eval and utils.graph